# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Use pandas.read_html() to extract tables from web page

In [2]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)

In [3]:
len(df_list)

3

# The table of interest is the first one 

In [4]:
tor_pc = df_list[0]

# Select only rows where ['Borough'] != 'Not assigned']
## Note that Postal Codes are already unique in the data

In [5]:
tor_df = tor_pc[tor_pc['Borough'] != 'Not assigned']

In [6]:
tor_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
tor_df.shape

(103, 3)

# Attempt to geocode: not working

In [9]:
#g = geocoder.google('M3A, Toronto, Ontario')

# Get CSV and join it

In [10]:
tor_ll = pd.read_csv('/home/john/Downloads/Coursera/Geospatial_Coordinates.csv')


In [11]:
tor_join =tor_df.join(tor_ll, lsuffix='_caller', rsuffix='_other').drop(columns='Postal Code_other')


In [12]:
tor_join.rename(columns={"Postal Code_caller": "Postal Code"}, inplace=True)

# Clean data and select Toronto City neignbourhoods

In [13]:
tor = tor_join.dropna(axis=0, how='any')
tor_city = tor[tor['Borough'].str.contains('Toronto')]
tor_city

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.781638,-79.304302
22,M5C,Downtown Toronto,St. James Town,43.770120,-79.408493
30,M4E,East Toronto,The Beaches,43.737473,-79.464763
31,M5E,Downtown Toronto,Berczy Park,43.739015,-79.506944
40,M5G,Downtown Toronto,Central Bay Street,43.685347,-79.338106
41,M6G,Downtown Toronto,Christie,43.679557,-79.352188
49,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.686412,-79.400049
50,M6H,West Toronto,"Dufferin, Dovercourt Village",43.679563,-79.377529


# Set Foursquare credentials

In [14]:
CLIENT_ID = 'FBUSMB1E55HUZ4HCO5GMQKSOVBXDXEGACXAE4PPBDIS1DFSN' # your Foursquare ID
CLIENT_SECRET = 'X55DLMCA0J1NCGMUPOZW2QGBRJAYJUPFR2SYU20X5BLR3BB1' # your Foursquare Secret
ACCESS_TOKEN = 'MCTBZD2MR32PYRPMDWQMXWV1UKWKAMOG1WSUEOUL4UMGKZ0H' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FBUSMB1E55HUZ4HCO5GMQKSOVBXDXEGACXAE4PPBDIS1DFSN
CLIENT_SECRET:X55DLMCA0J1NCGMUPOZW2QGBRJAYJUPFR2SYU20X5BLR3BB1


In [22]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[43.667856,-79.532242], zoom_start=10)

# add markers to map
for lat, lng, label in zip(tor_city['Latitude'], tor_city['Longitude'], tor_city['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


# Test Foursquare search 

In [18]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=FBUSMB1E55HUZ4HCO5GMQKSOVBXDXEGACXAE4PPBDIS1DFSN&client_secret=X55DLMCA0J1NCGMUPOZW2QGBRJAYJUPFR2SYU20X5BLR3BB1&v=20180604&ll=43.706748299999994,-79.5940544&radius=500&limit=100'

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Find up to 100 venues around each postal code 

In [28]:
toronto_venues = getNearbyVenues(names=tor_city['Neighbourhood'],
                                   latitudes=tor_city['Latitude'],
                                   longitudes=tor_city['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North


In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(513, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
1,"Regent Park, Harbourfront",43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
2,"Regent Park, Harbourfront",43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
3,"Regent Park, Harbourfront",43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports
4,"Regent Park, Harbourfront",43.773136,-79.239476,TD Canada Trust,43.774830,-79.241251,Bank


## Count number of venues found (Only Little Portugal, Trinity found the max 100)

In [30]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,6,6,6,6,6,6
"Brockton, Parkdale Village, Exhibition Place",18,18,18,18,18,18
Central Bay Street,4,4,4,4,4,4
Christie,42,42,42,42,42,42
"Commerce Court, Victoria Hotel",17,17,17,17,17,17
Davisville North,5,5,5,5,5,5
"Dufferin, Dovercourt Village",4,4,4,4,4,4
"Garden District, Ryerson",15,15,15,15,15,15
"Harbourfront East, Union Station, Toronto Islands",93,93,93,93,93,93


In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 172 uniques categories.


In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Drugstore,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,History Museum,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0

In [162]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()


In [163]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
           venue  freq
0  Grocery Store  0.33
1  Shopping Mall  0.17
2           Park  0.17
3          Hotel  0.17
4           Bank  0.17


----Brockton, Parkdale Village, Exhibition Place----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                     venue  freq
0                     Park  0.50
1            Metro Station  0.25
2        Convenience Store  0.25
3  New American Restaurant  0.00
4       Mexican Restaurant  0.00


----Christie----
                    venue  freq
0        Greek Restaurant  0.17
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----Commerce Court, Victoria Hotel----
                       venue  freq
0                   Pharmacy  0.12
1                     Bakery  0.12
2                       Pool  0.06
3           

In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Grocery Store,Hotel,Bank,Shopping Mall,Park,College Gym,Department Store,Dumpling Restaurant,Drugstore,Doner Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Boat or Ferry,Sculpture Garden,Rental Car Location,Bar,Plane
2,Central Bay Street,Park,Metro Station,Convenience Store,Yoga Studio,Deli / Bodega,Drugstore,Doner Restaurant,Discount Store,Diner,Dessert Shop
3,Christie,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Yoga Studio,Fruit & Vegetable Store,Pizza Place,Lounge
4,"Commerce Court, Victoria Hotel",Bakery,Pharmacy,Grocery Store,Furniture / Home Store,Middle Eastern Restaurant,Music Venue,Park,Café,Pool,Brewery


In [153]:
# set number of clusters
kclusters =5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 1, 2, 2, 2, 1, 2, 2, 3], dtype=int32)

In [154]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = tor_city

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_clean = toronto_merged.dropna() 



In [155]:
# create map
map_clusters = folium.Map(location=[43.7,-79.40000], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clean['Latitude'], toronto_clean['Longitude'], toronto_clean['Neighbourhood'], toronto_clean['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [156]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 0, toronto_clean.columns[[2] + list(range(5, toronto_clean.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Roselawn,0.0,Filipino Restaurant,Bakery,Yoga Studio,Event Space,Dumpling Restaurant,Drugstore,Doner Restaurant,Discount Store,Diner,Dessert Shop


In [157]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 1, toronto_clean.columns[[2] + list(range(5, toronto_clean.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Central Bay Street,1.0,Park,Metro Station,Convenience Store,Yoga Studio,Deli / Bodega,Drugstore,Doner Restaurant,Discount Store,Diner,Dessert Shop
50,"Dufferin, Dovercourt Village",1.0,Park,Playground,Trail,Yoga Studio,Cupcake Shop,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store


In [158]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 2, toronto_clean.columns[[2] + list(range(5, toronto_clean.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Regent Park, Harbourfront",2.0,Lounge,Hakka Restaurant,Gas Station,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Bakery,Deli / Bodega
6,"Queen's Park, Ontario Provincial Government",2.0,Coffee Shop,Chinese Restaurant,Bus Station,Convenience Store,Department Store,Deli / Bodega,Dumpling Restaurant,Drugstore,Doner Restaurant,Discount Store
13,"Garden District, Ryerson",2.0,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Thai Restaurant,Shopping Mall,Rental Car Location,Italian Restaurant,Intersection,Bank,Convenience Store
22,St. James Town,2.0,Ramen Restaurant,Pizza Place,Shopping Mall,Sandwich Place,Restaurant,Sushi Restaurant,Coffee Shop,Café,Lounge,Plaza
41,Christie,2.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Yoga Studio,Fruit & Vegetable Store,Pizza Place,Lounge
49,"Richmond, Adelaide, King",2.0,Coffee Shop,Bagel Shop,Athletics & Sports,Bank,Supermarket,Sushi Restaurant,Pub,Fried Chicken Joint,Pizza Place,American Restaurant
58,"Harbourfront East, Union Station, Toronto Islands",2.0,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym,Thai Restaurant,Clothing Store,Pizza Place,Concert Hall,Lounge
59,"Little Portugal, Trinity",2.0,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Italian Restaurant,Plaza
66,"The Danforth West, Riverdale",2.0,Café,Bakery,Bookstore,Japanese Restaurant,Bar,Yoga Studio,College Arts Building,College Gym,Comfort Food Restaurant,Dessert Shop
67,"Toronto Dominion Centre, Design Exchange",2.0,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Grocery Store,Bakery,Caribbean Restaurant,Bar,Gaming Cafe


In [159]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 3, toronto_clean.columns[[2] + list(range(5, toronto_clean.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Berczy Park,3.0,Grocery Store,Hotel,Bank,Shopping Mall,Park,College Gym,Department Store,Dumpling Restaurant,Drugstore,Doner Restaurant
75,"India Bazaar, The Beaches West",3.0,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Candy Store,Restaurant,Italian Restaurant,Baby Store,Coffee Shop


In [160]:
toronto_clean.loc[toronto_clean['Cluster Labels'] == 4, toronto_clean.columns[[2] + list(range(5, toronto_clean.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,The Beaches,4.0,Airport,Snack Place,Park,Dance Studio,Drugstore,Doner Restaurant,Discount Store,Diner,Dessert Shop,Department Store
